In [6]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from scipy.spatial import procrustes
from PIL import Image, ImageDraw, ImageFont

In [7]:
def process_image_for_faces(image_path):
    app = FaceAnalysis(allowed_modules=["detection", "landmark_2d_106"], providers=["CUDAExecutionProvider"])
    app.prepare(ctx_id=0, det_size=(640, 640))

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or unable to load.")
    
    faces_data = app.get(img)
    faces_landmarks = []
    bounding_boxes = []
    
    for face in faces_data:
        if "landmark_2d_106" in face:
            lmk = face["landmark_2d_106"]
            bbox = face["bbox"].astype(np.int32)
            faces_landmarks.append(np.round(lmk).astype(np.int64))
            bounding_boxes.append(bbox)
    
    return faces_landmarks, bounding_boxes, img

In [8]:
def calculate_distances(landmarks):
    distances = {}
    # 눈과 눈썹 사이의 거리
    distances['eye_to_eyebrow_left'] = np.linalg.norm(landmarks[37] - landmarks[19])
    distances['eye_to_eyebrow_right'] = np.linalg.norm(landmarks[46] - landmarks[24])

    # 눈의 가로 길이
    distances['eye_width_left'] = np.linalg.norm(landmarks[36] - landmarks[39])
    distances['eye_width_right'] = np.linalg.norm(landmarks[42] - landmarks[45])

    # 코의 길이
    distances['nose_length'] = np.linalg.norm(landmarks[30] - landmarks[33])

    # 입과 코 사이의 거리
    distances['nose_to_mouth'] = np.linalg.norm(landmarks[33] - landmarks[51])

    # 입술의 너비
    distances['mouth_width'] = np.linalg.norm(landmarks[48] - landmarks[54])

    # 이마의 높이 (이마 중앙에서 눈썹 중앙까지)
    distances['forehead_height'] = np.linalg.norm(landmarks[27] - landmarks[8])

    # 턱선의 길이 (턱 끝에서 귀 바로 아래까지)
    distances['jawline_length_left'] = np.linalg.norm(landmarks[5] - landmarks[8])
    distances['jawline_length_right'] = np.linalg.norm(landmarks[11] - landmarks[8])

    # 이마의 너비 (이마 양 끝)
    distances['forehead_width'] = np.linalg.norm(landmarks[0] - landmarks[16])

    # 얼굴의 높이 (이마 중앙에서 턱 끝까지)
    distances['face_height'] = np.linalg.norm(landmarks[27] - landmarks[8])

    return distances


In [9]:
def compare_faces_landmarks(faces_landmarks):
    results = []
    print("Procrustes disparity results for all possible face matches:")
    for idx1, lmk1 in enumerate(faces_landmarks):
        for idx2, lmk2 in enumerate(faces_landmarks):
            if idx1 != idx2:  # 자기 자신과의 비교를 제외
                mtx1, mtx2, disparity = procrustes(lmk1, lmk2)
                results.append((idx1, idx2, disparity))
                print(f"Face {idx1} vs Face {idx2}: Disparity = {disparity:.4f}")
    return results

In [10]:
def compare_all_features(landmarks1, landmarks2):
    distances1 = calculate_distances(landmarks1)
    distances2 = calculate_distances(landmarks2)
    results = {}
    for key in distances1:
        # 각 특징 거리 차이를 비교
        disparity = abs(distances1[key] - distances2[key])
        results[key] = disparity
    return results


In [11]:
def draw_matching_faces(image1, landmarks1, bounding_boxes1, image2, landmarks2, bounding_boxes2, matches):
    pil_image1 = Image.fromarray(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
    pil_image2 = Image.fromarray(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
    draw1 = ImageDraw.Draw(pil_image1)
    draw2 = ImageDraw.Draw(pil_image2)

    # Load a font
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except IOError:
        font = ImageFont.load_default()

    color = (255, 0, 0)  # Set a color for the bounding box and landmarks
    point_radius = 2
    
    for idx1, (landmarks, bbox) in enumerate(zip(landmarks1, bounding_boxes1)):
        for point in landmarks:
            draw1.ellipse([point[0]-point_radius, point[1]-point_radius, point[0]+point_radius, point[1]+point_radius], fill=color)
        draw1.rectangle([bbox[0], bbox[1], bbox[2], bbox[3]], outline=color, width=2)
        draw1.text((bbox[0], bbox[1] - 20), f"Face {idx1}", fill=(255, 255, 255), font=font)
        
    for idx2, (landmarks, bbox) in enumerate(zip(landmarks2, bounding_boxes2)):
        for point in landmarks:
            draw2.ellipse([point[0]-point_radius, point[1]-point_radius, point[0]+point_radius, point[1]+point_radius], fill=color)
        draw2.rectangle([bbox[0], bbox[1], bbox[2], bbox[3]], outline=color, width=2)
        draw2.text((bbox[0], bbox[1] - 20), f"Face {idx2}", fill=(255, 255, 255), font=font)

    # Highlight matching faces with a different color
    match_color = (0, 255, 0)
    for match in matches:
        idx1, idx2, disparity = match
        if disparity < 0.05:  # Threshold for matching, can be adjusted
            bbox1 = bounding_boxes1[idx1]
            bbox2 = bounding_boxes2[idx2]
            draw1.rectangle([bbox1[0], bbox1[1], bbox1[2], bbox1[3]], outline=match_color, width=3)
            draw2.rectangle([bbox2[0], bbox2[1], bbox2[2], bbox2[3]], outline=match_color, width=3)

    pil_image1.show()
    pil_image2.show()

In [12]:
# Paths to the images
img_path1 = "data/aespa1.jpg"
img_path2 = "data/aespa2.jpg"

landmarks1, boxes1, img1 = process_image_for_faces(img_path1)
landmarks2, boxes2, img2 = process_image_for_faces(img_path2)

if landmarks1 and landmarks2:
    comparison_results = compare_faces_landmarks(landmarks1, landmarks2)
    draw_matching_faces(img1, landmarks1, boxes1, img2, landmarks2, boxes2, comparison_results)
else:
    print("Landmarks were not detected in one or both images.")

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hancomtst/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hancomtst/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\w600k_r50.onnx recognition
set det-size: (640, 640)
Applied providers: ['CPUExecution

TypeError: compare_faces_landmarks() takes 1 positional argument but 2 were given

In [13]:
features_comparison = compare_all_features(landmarks1[0], landmarks2[0])
print("Feature disparities between two faces:")
for feature, disparity in features_comparison.items():
    print(f"{feature}: {disparity:.4f}")

Feature disparities between two faces:
eye_to_eyebrow_left: 10.1794
eye_to_eyebrow_right: 16.8172
eye_width_left: 0.5176
eye_width_right: 0.0000
nose_length: 6.8069
nose_to_mouth: 0.9443
mouth_width: 8.0474
forehead_height: 14.5859
jawline_length_left: 3.8925
jawline_length_right: 13.0166
forehead_width: 8.1880
face_height: 14.5859


In [15]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from PIL import Image, ImageDraw, ImageFont

def process_image_for_faces(image_path):
    app = FaceAnalysis(allowed_modules=["detection", "landmark_2d_106"], providers=["CUDAExecutionProvider"])
    app.prepare(ctx_id=0, det_size=(640, 640))

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or unable to load.")
    
    faces_data = app.get(img)
    faces_landmarks = []
    bounding_boxes = []
    
    for face in faces_data:
        lmk = face["landmark_2d_106"]
        bbox = face["bbox"].astype(np.int32)
        faces_landmarks.append(np.round(lmk).astype(np.int64))
        bounding_boxes.append(bbox)
    
    return faces_landmarks, bounding_boxes, img

def calculate_distances(landmarks):
    distances = {
        'eyebrow_length_left': np.linalg.norm(landmarks[46] - landmarks[43]),
        'eyebrow_length_right': np.linalg.norm(landmarks[97] - landmarks[101]),
        'eye_width_left': np.linalg.norm(landmarks[39] - landmarks[35]),
        'eye_width_right': np.linalg.norm(landmarks[89] - landmarks[93]),
        'nose_length': np.linalg.norm(landmarks[72] - landmarks[86]),
        'nose_to_mouth': np.linalg.norm(landmarks[80] - landmarks[71]),
        'mouth_width': np.linalg.norm(landmarks[52] - landmarks[61]),
        'jawline_length_left': np.linalg.norm(landmarks[1] - landmarks[0]),
        'jawline_length_right': np.linalg.norm(landmarks[17] - landmarks[0])
    }
    return distances

def compare_all_faces_features(faces_landmarks1, faces_landmarks2):
    all_results = {}
    for idx1, lmk1 in enumerate(faces_landmarks1):
        face_results = []
        for idx2, lmk2 in enumerate(faces_landmarks2):
            distances1 = calculate_distances(lmk1)
            distances2 = calculate_distances(lmk2)
            disparities = {key: abs(distances1[key] - distances2[key]) for key in distances1}
            similarity = calculate_similarity(disparities)
            face_results.append(similarity)
            print(f"Similarity between Face {idx1} in Image 1 and Face {idx2} in Image 2: {similarity:.4f}")
        all_results[idx1] = face_results
    return all_results

def calculate_similarity(disparities):
    mean_disparity = np.mean(list(disparities.values()))
    similarity = 1 - (mean_disparity / 10)
    return max(0, min(similarity, 1))

# Define the paths to the images
img_path1 = "data/aespa1.jpg"
img_path2 = "data/aespa2.jpg"

# Process the images and extract landmarks and bounding boxes
landmarks1, boxes1, img1 = process_image_for_faces(img_path1)
landmarks2, boxes2, img2 = process_image_for_faces(img_path2)

# Compare all faces across the two images if landmarks are detected
if landmarks1 and landmarks2:
    comparison_results = compare_all_faces_features(landmarks1, landmarks2)
    for face_idx, similarities in comparison_results.items():
        max_similarity = max(similarities)
        print(f"Highest similarity for Face {face_idx} in Image 1: {max_similarity:.4f}")
else:
    print("Landmarks were not detected in one or both images.")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hancomtst/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hancomtst/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\w600k_r50.onnx recognition
set det-size: (640, 640)
Applied providers: ['CPUExecution

In [16]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from PIL import Image, ImageDraw, ImageFont

def process_image_for_faces(image_path):
    app = FaceAnalysis(allowed_modules=["detection", "landmark_2d_106"], providers=["CUDAExecutionProvider"])
    app.prepare(ctx_id=0, det_size=(640, 640))

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or unable to load.")
    
    faces_data = app.get(img)
    faces_landmarks = []
    bounding_boxes = []
    
    for face in faces_data:
        lmk = face["landmark_2d_106"]
        bbox = face["bbox"].astype(np.int32)
        faces_landmarks.append(np.round(lmk).astype(np.int64))
        bounding_boxes.append(bbox)
    
    return faces_landmarks, bounding_boxes, img

def draw_faces_with_landmarks(image, faces_landmarks, bounding_boxes):
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_image)

    # Load a font, or use PIL's default if unavailable
    try:
        font = ImageFont.truetype("arial.ttf", 10)
    except IOError:
        font = ImageFont.load_default()

    # Draw each face's landmarks and bounding box
    for idx, (landmarks, bbox) in enumerate(zip(faces_landmarks, bounding_boxes)):
        draw.rectangle([bbox[0], bbox[1], bbox[2], bbox[3]], outline="green", width=2)
        draw.text((bbox[0], bbox[1] - 10), f"Face {idx}", fill="yellow", font=font)
        for point_idx, point in enumerate(landmarks):
            point_pos = (point[0] - 2, point[1] - 2, point[0] + 2, point[1] + 2)
            draw.ellipse(point_pos, fill="red")
            draw.text((point[0] + 4, point[1]), str(point_idx), fill="blue", font=font)

    pil_image.show()

# Define the paths to the images
img_path1 = "data/winter1.jpg"

# Process the image and extract landmarks and bounding boxes
landmarks1, boxes1, img1 = process_image_for_faces(img_path1)

# Draw landmarks on the image
draw_faces_with_landmarks(img1, landmarks1, boxes1)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hancomtst/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hancomtst/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\w600k_r50.onnx recognition
set det-size: (640, 640)
